## The Office character prediction by line

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import PassiveAggressiveClassifier, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MaxAbsScaler, Normalizer
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Embedding, Dropout

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/the-office-us-complete-dialoguetranscript/The-Office-Lines-V4.csv')

## Feature engineering

In [ ]:
data

In [ ]:
count = Counter(data['speaker'])
names = []
X = []
y = []

Remove all characters that have less than 1000 lines, so we can focus only on main characters

In [ ]:
i = 0
for j in count.keys():
    if list(count.values())[i] > 1000:
        names.append(list(count.keys())[i])
    i += 1

In [ ]:
i = 0
while i < len(data['speaker']):
    if data['speaker'][i] in names:
        X.append(data['line'][i])
        y.append(data['speaker'][i])
    i += 1


In [ ]:
for i in range(10):
    print(y[i].ljust(15) + X[i])

In [ ]:
num_classes = np.unique(y).shape[0]
num_classes

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import argmax
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [ ]:
np.array(onehot_encoded).shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, onehot_encoded, test_size=0.2, random_state=1)

In [ ]:
max_words = 1000

In [ ]:

tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', oov_token='unknown', char_level=False)

tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index.items())

In [ ]:
X_train_tokenized = tokenizer.texts_to_matrix(X_train)
X_test_tokenized = tokenizer.texts_to_matrix(X_test)

## Visualize most common words

In [ ]:
wordcloud = WordCloud(background_color='white').generate(' '.join(X))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
X_train_tokenized

In [ ]:
X_test_tokenized

In [ ]:
model01 = Sequential()
model01.add(Dense(100, input_dim=max_words, activation="relu"))
model01.add(Dropout(0.4))
model01.add(Dense(100, activation='relu'))
model01.add(Dropout(0.4))
model01.add(Dense(100, activation='relu'))
model01.add(Dropout(0.4))
model01.add(Dense(num_classes, activation='softmax'))

model01.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#Обучаем сеть на выборке
history = model01.fit(X_train_tokenized, 
                    y_train, 
                    epochs=5,
                    batch_size=128,
                    validation_split=0.1)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
model01.evaluate(X_test_tokenized, y_test)

29% > 9%, good enough.